In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pawankumargunjan/imdb-review")

print("Path to dataset files:", path)

100%|██████████| 111M/111M [00:05<00:00, 23.1MB/s] 

Extracting files...


Path to dataset files: C:\Users\orlov\.cache\kagglehub\datasets\pawankumargunjan\imdb-review\versions\3


In [7]:
path = f'{path}\\aclImdb'

In [9]:
import os
import pandas as pd

def load_reviews(base_path):
    reviews = []
    labels = []
    
    for label in ['pos', 'neg']:
        folder_path = os.path.join(base_path, label)
        print(folder_path)
        for filename in os.listdir(folder_path):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                reviews.append(file.read())
                labels.append(1 if label == 'pos' else 0)  # 1 для позитивных, 0 для негативных

    return pd.DataFrame({'review': reviews, 'sentiment': labels})

data_train = load_reviews(f'{path}\\train')
data_test = load_reviews(f'{path}\\test')


C:\Users\orlov\.cache\kagglehub\datasets\pawankumargunjan\imdb-review\versions\3\aclImdb\train\pos
C:\Users\orlov\.cache\kagglehub\datasets\pawankumargunjan\imdb-review\versions\3\aclImdb\train\neg
C:\Users\orlov\.cache\kagglehub\datasets\pawankumargunjan\imdb-review\versions\3\aclImdb\test\pos
C:\Users\orlov\.cache\kagglehub\datasets\pawankumargunjan\imdb-review\versions\3\aclImdb\test\neg


In [10]:
data_test

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
24995,I occasionally let my kids watch this garbage ...,0
24996,When all we have anymore is pretty much realit...,0
24997,The basic genre is a thriller intercut with an...,0
24998,Four things intrigued me as to this film - fir...,0


---

Буду честен - гпт подсказала использовать для векторизации TF-IDF. Ниже его принцип:

TF-IDF (Term Frequency-Inverse Document Frequency): Это метод векторизации
Term Frequency (TF): Частота термина в документе.
Inverse Document Frequency (IDF): Мера, показывающая важность термина в общем наборе документов. Чем реже встречается слово, тем выше его вес.
В результате получаем матрицу, где строки — это отзывы, а столбцы — это уникальные слова, а значения — это весовые коэффициенты.

Случайный же лес взял, поскольку показалось самым точным и понятным решением. Не очень хотел использовать регрессию, поскольку скорее всего отзывы не будут линейно связаны с оценкой. Например даже первый отзыв вроде бы использует много формулировок, которые сходу можно назвать негативными, однако, отзыв помечен, как положительный и при прочтении он действительно положительный - человек удивился, что режиссер снял что-то хорошее, кроме комедий. Поэтому решил попробовать случайный лес.

Сам случайный лес - набор деревьев решений, каждое из которых, по сути, является блоком сравнения, параметр сравнения которого меняется в процессе обучения в нужную сторону.

Вот еще пара формулировок от ГПТ:
Случайный выбор признаков: На каждом узле дерева случайным образом выбирается подмножество признаков, чтобы определить, какой признак использовать для разделения. Это добавляет разнообразие между деревьями и уменьшает вероятность переобучения.

Объединение предсказаний: Для классификации итоговое предсказание формируется путем голосования большинства (каждое дерево отдает голос за свой класс), а для регрессии — путем усреднения предсказаний всех деревьев.

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Создание и обучение модели случайного леса
model = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=100, random_state=42))
model.fit(data_train['review'], data_train['sentiment'])


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [12]:
# Предсказание на тестовых данных
y_pred = model.predict(data_test['review'])

# Отчет о качестве модели
print(classification_report(data_test['sentiment'], y_pred, target_names=['Negative', 'Positive']))


              precision    recall  f1-score   support

    Negative       0.83      0.85      0.84     12500
    Positive       0.85      0.83      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



Судя по данным, моделька справилась довольно сносно - 84 % случаев предсказания оказались верными)

In [15]:
from sklearn.metrics import mean_squared_error as MSE

MSE тоже неплохой - 0.16. Учитывая масштаб наших данных, я бы сказал, что можно и лучше, но и это довольно хорошо. Примерно это означает, что мы правы в 84 случаях из 100, что достаточно круто

In [16]:
MSE(data_test['sentiment'], y_pred)

np.float64(0.16172)

Вот и пример отзыва и его интерпретация

In [17]:
y_pred[0]

np.int64(1)

In [14]:
data_test['review'][0]

"I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge."